# Dashboard de Segmentación de Clientes con RFM, Clusters y Storytelling

Este dashboard ofrece una visión integral de la segmentación de clientes basada en RFM, patrones de consumo (topics), clustering y recomendaciones accionables. Incluye visualizaciones comparativas, tabla filtrable/exportable y predicción para nuevos clientes.

## Corrección importante
> **Nota:** Se corrige el error `TypeError: Object of type Period is not JSON serializable` convirtiendo toda columna de tipo `Period` (como 'month') a string antes de usarla en visualizaciones/tables.

In [1]:
# Importación de librerías
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, NMF
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc, html, Input, Output, State, dash_table
import dash_bootstrap_components as dbc
from scipy.stats import entropy
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

## 1. Carga de datos y KPIs

In [2]:
file_path = "Data_Set_Global.xlsx"
pedidos = pd.read_excel(file_path, sheet_name="Pedidos")
pedidos["total_price"] = pedidos["quantity"] * pedidos["unit_price"]
pedidos["order_date"] = pd.to_datetime(pedidos["order_date"])
pedidos["promised_delivery_time"] = pd.to_datetime(pedidos["promised_delivery_time"])
pedidos["actual_delivery_time"] = pd.to_datetime(pedidos["actual_delivery_time"])
pedidos["delivery_delay_min"] = (
    (pedidos["actual_delivery_time"] - pedidos["promised_delivery_time"]).dt.total_seconds() / 60
)
pedidos = pedidos.dropna(subset=["order_id", "customer_id", "order_date"]).copy()
kpi_dict = {
    "Total Clientes": pedidos["customer_id"].nunique(),
    "Total Órdenes": pedidos["order_id"].nunique(),
    "Total Productos": pedidos["product_id"].nunique(),
    "Facturación Total": round(pedidos["total_price"].sum(),2),
    "Periodo Analizado": f"{pedidos['order_date'].min().date()} a {pedidos['order_date'].max().date()}"
}

## 2. Análisis Exploratorio Visual (EDA)

In [3]:
df_seg = pedidos.groupby("customer_segment")[["order_id"]].count().reset_index()
fig_bar_segment = px.bar(df_seg, x="customer_segment", y="order_id", text="order_id",
                        labels={"order_id": "# Órdenes"}, title="Órdenes por Segmento de Cliente")
N = 5
topN = (
    pedidos.groupby(["customer_segment", "product_name"])
    .size().groupby(level=0, group_keys=False).nlargest(N)
    .reset_index(name="compras")
)
top5_prod_segment = {}
for seg in topN["customer_segment"].unique():
    top5_prod_segment[seg] = topN[topN["customer_segment"] == seg][["product_name", "compras"]].to_dict('records')
num_cols = pedidos.select_dtypes("number").columns
corr = pedidos[num_cols].corr().round(2)
fig_corr = px.imshow(corr, color_continuous_scale='RdBu', title="Matriz de Correlación Variables Numéricas")
fig_delay = px.histogram(pedidos, x="delivery_delay_min", nbins=50, title="Demora de entrega (min)")
agg = (
    pedidos.groupby("customer_id")
    .agg(
        first_order=("order_date", "min"),
        registration_date=("registration_date", "first")
    ).reset_index()
)
agg["days_to_first"] = (agg["first_order"] - agg["registration_date"]).dt.days
fig_conversion = px.histogram(agg, x="days_to_first", nbins=30, title="Días hasta primera compra")

## 3. Variables RFM, Topics y Clustering

In [4]:
snapshot_date = pedidos["order_date"].max() + dt.timedelta(days=1)
rfm = pedidos.groupby("customer_id").agg(
    recency=("order_date", lambda date: (snapshot_date - date.max()).days),
    frequency=("order_id", "nunique"),
    monetary=("total_price", "sum")
)
scaler_rfm = StandardScaler()
rfm_z = pd.DataFrame(scaler_rfm.fit_transform(rfm), index=rfm.index, columns=[c+"_z" for c in rfm.columns])
pay_counts = pedidos.groupby(["customer_id", "payment_method"]).size().unstack(fill_value=0)
pay_probs = pay_counts.div(pay_counts.sum(axis=1), axis=0)
payment_entropy = pay_probs.apply(lambda row: entropy(row, base=2), axis=1)
customer_payment_diversity = payment_entropy.rename("payment_entropy").reset_index()
cust_prod = pedidos.groupby(["customer_id", "product_id"]).size().unstack(fill_value=0)
K = 10
nmf = NMF(n_components=K, init="random", random_state=42)
W = nmf.fit_transform(cust_prod)
H = nmf.components_
customer_topics = pd.DataFrame(W, index=cust_prod.index, columns=[f"topic_{i+1}" for i in range(K)])
full_df = (
    rfm_z
    .join(customer_payment_diversity.set_index("customer_id"))
    .join(customer_topics)
).fillna(0)
X_scaled = StandardScaler().fit_transform(full_df.select_dtypes(include=np.number))
product_ids = cust_prod.columns
prod_meta = pd.read_excel(file_path, sheet_name="Productos").set_index("product_id")
top_words_topic = {}
for i, topic in enumerate(H):
    top_ids = np.argsort(topic)[-5:][::-1]
    top_names = prod_meta.loc[product_ids[top_ids], "product_name"].tolist()
    top_words_topic[f"topic_{i+1}"] = top_names
# Clustering
K_OPT = 3
kmeans = KMeans(n_clusters=K_OPT, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_scaled)
full_df["cluster"] = cluster_labels
full_df.index.name = "customer_id"
profile_cols = [c for c in full_df.columns if c.startswith(('recency','frequency','monetary','payment_entropy','topic_'))]
cluster_profile = (
    full_df.groupby("cluster")[profile_cols].mean().round(2)
)

  File "C:\Users\aresu\ANACONDA\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\aresu\ANACONDA\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\aresu\ANACONDA\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\aresu\ANACONDA\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


## 4. Visualización de clusters y radar comparativo

In [5]:
pca = PCA(n_components=2)
coords = pca.fit_transform(X_scaled)
df_plot = full_df.reset_index().copy()
df_plot['PCA1'] = coords[:,0]
df_plot['PCA2'] = coords[:,1]
fig_pca = px.scatter(
    df_plot, x='PCA1', y='PCA2', color=df_plot['cluster'].astype(str),
    title="Visualización de Clusters (PCA)",
    labels={"color": "Cluster"}, hover_data=['customer_id']
)
def radar_cluster(cluster_id, cluster_profile):
    variables = ['recency_z', 'frequency_z', 'monetary_z', 'payment_entropy']
    vals = cluster_profile.loc[cluster_id, variables].tolist()
    fig = go.Figure(data=go.Scatterpolar(
        r=vals + [vals[0]],
        theta=[v.replace('_z','').capitalize() for v in variables] + [variables[0].replace('_z','').capitalize()],
        fill='toself', name=f"Cluster {cluster_id}"
    ))
    fig.update_layout(polar=dict(radialaxis=dict(visible=True)), showlegend=False,
                      title=f"Perfil del Cluster {cluster_id}")
    return fig

## 5. Diccionario de clusters, topics y recomendaciones

In [6]:
cluster_names = {
    0: "Champions conveniencia",
    1: "Dormant básicos",
    2: "Compradores Ocasionales"
}
topic_explainer = {
    "topic_1": {
        "categorias": "Infant Hygiene, Cooking Essentials, Pet Snacks",
        "nombre": "Cuidado familiar & básicos",
        "razon": "Mezcla de higiene infantil y despensa esencial con algo de snacks de mascota."
    },
    "topic_2": {
        "categorias": "Pet Snacks, Bakery, Home Cleaning",
        "nombre": "Mascotas & panadería",
        "razon": "Fuerte en snacks de mascota y panificados; limpieza como apoyo del hogar."
    },
    "topic_6": {
        "categorias": "Baby Food, Baby Wipes, Tomatoes",
        "nombre": "Dulces & limpieza",
        "razon": "Golosinas acompañadas de artículos de aseo."
    }
}
def cluster_recommendation(cluster_id):
    recs = {
        0: "Enfocar campañas de lealtad y venta cruzada de productos premium.",
        1: "Recuperar clientes inactivos con promociones personalizadas.",
        2: "Explotar ventas de impulsos y combos, y fomentar la recompra."
    }
    return html.Div([
        html.H5("Recomendación de Negocio:"),
        html.P(recs.get(cluster_id, "Analizar el comportamiento de este segmento."))
    ], style={"backgroundColor":"#f0f0f0", "padding":"1em", "borderRadius":"8px"})
def cluster_summary(cluster_id, cluster_profile, top_words_topic, cluster_names, topic_explainer):
    row = cluster_profile.loc[cluster_id]
    name = cluster_names.get(cluster_id, f"Cluster {cluster_id}")
    topic_cols = [col for col in cluster_profile.columns if col.startswith('topic_')]
    top_topic = row[topic_cols].sort_values(ascending=False).head(1)
    topic_name = top_topic.index[0]
    topic_value = top_topic.iloc[0]
    topic_info = topic_explainer.get(topic_name, {})
    categorias = topic_info.get("categorias", ", ".join(top_words_topic.get(topic_name, [])))
    nombre_topic = topic_info.get("nombre", topic_name)
    razon = topic_info.get("razon", "")
    resumen = f"""
## Cluster {cluster_id}: {name}

**¿Qué significa cada métrica?**
- **Recency (Z):** {row['recency_z']:.2f}. Un valor más bajo indica que los clientes compraron más recientemente (clientes activos).
- **Frequency (Z):** {row['frequency_z']:.2f}. Un valor más alto indica mayor frecuencia de compra.
- **Monetary (Z):** {row['monetary_z']:.2f}. Un valor alto señala que estos clientes gastan más que el promedio.
- **Diversidad de pago:** {row['payment_entropy']:.2f}. Un valor alto sugiere clientes flexibles que usan varios métodos de pago.

**Tópico/patrón principal:**
- {topic_name} (afinidad {topic_value:.2f})
- **Categorías dominantes:** {categorias}
- **Nombre sugerido:** {nombre_topic}
- **Razonamiento breve:** {razon}

**Importancia del número de cluster:**
- El número de cluster permite identificar segmentos únicos y facilita la toma de decisiones personalizadas para cada grupo. Debe ser destacado en todos los informes y visualizaciones.
"""
    return resumen

## 6. Comparativa de KPIs y evolución temporal de clusters

🔵 **¡Aquí corregimos el error de serialización!**
Convertimos la columna 'month' a string antes de graficar.

In [7]:
# Comparativa de KPIs entre clusters
fig_kpi = px.bar(
    cluster_profile.reset_index().melt(id_vars='cluster', value_vars=['recency_z','frequency_z','monetary_z','payment_entropy']),
    x='variable', y='value', color='cluster', barmode='group',
    labels={'value':'Valor Z','variable':'KPI','cluster':'Cluster'},
    title="KPIs promedio por Cluster"
)
# Evolución temporal de tamaño de clusters
pedidos = pedidos.merge(full_df['cluster'], left_on='customer_id', right_index=True)
pedidos['month'] = pedidos['order_date'].dt.to_period('M').astype(str)  # <--- CORRECCIÓN AQUÍ
evol = pedidos.groupby(['month','cluster'])['customer_id'].nunique().reset_index()
fig_evol = px.line(evol, x='month', y='customer_id', color='cluster', markers=True,
                   labels={'customer_id':'N° clientes','month':'Mes','cluster':'Cluster'},
                   title="Evolución del tamaño de cada cluster")

## 7. Layout Dash con tabla filtrable/exportable, visuales y predicción

In [8]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.MINTY])
app.layout = dbc.Container([
    html.H2("Dashboard de Segmentación de Clientes: Explicación RFM y Topics", style={"marginTop": 15}),
    html.P("Este dashboard integra el análisis RFM, clustering y topics de productos para explicar y accionar sobre los segmentos de clientes."),
    html.Hr(),
    html.H4("¿Qué significa el análisis RFM?"),
    html.P(
        "RFM segmenta clientes según Recency (recencia, cuánto tiempo desde la última compra; menor es mejor), "
        "Frequency (frecuencia, cuántas compras; mayor es mejor) y Monetary (monto gastado; mayor es mejor). "
        "Las puntuaciones Z indican si el cliente está por arriba o por debajo del promedio (0 = promedio). "
        "Diversidad de pago mide cuántos métodos de pago diferentes usa cada cliente."
    ),
    html.Hr(),
    html.H4("KPIs y contexto general"),
    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader(k),
                dbc.CardBody(html.H5(f"{v:,}" if isinstance(v, (int, float)) else str(v)))
            ]), width=3
        ) for k, v in kpi_dict.items()
    ]),
    html.Br(),
    html.H4("Análisis exploratorio visual"),
    dbc.Row([
        dbc.Col([dcc.Graph(figure=fig_bar_segment)], width=6),
        dbc.Col([dcc.Graph(figure=fig_corr)], width=6),
    ]),
    dbc.Row([
        dbc.Col([dcc.Graph(figure=fig_delay)], width=6),
        dbc.Col([dcc.Graph(figure=fig_conversion)], width=6),
    ]),
    html.Br(),
    html.H5("Top 5 productos por segmento de cliente"),
    html.Ul([
        html.Li([
            html.B(f"Segmento {seg}: "), 
            ', '.join([f"{d['product_name']} ({d['compras']})" for d in lst])
        ]) for seg, lst in top5_prod_segment.items()
    ]),
    html.Hr(),
    html.H4("Visualización de clusters (PCA)"),
    html.P("Cada punto es un cliente. Los colores muestran los clusters generados automáticamente. La distancia refleja similitud de patrones de compra."),
    dcc.Graph(figure=fig_pca),
    html.Hr(),
    html.H4("Perfil estratégico y explicación de clusters"),
    html.P("Selecciona un cluster para ver su explicación estratégica, métricas RFM, tópicos y razonamiento."),
    dcc.Dropdown(
        id='cluster-desc-dropdown',
        options=[{"label": f"{cluster_names[c]}", "value": c} for c in sorted(cluster_profile.index)],
        value=0, clearable=False
    ),
    html.Div(id='cluster-desc-panel', style={"whiteSpace": "pre-line", "padding": "1em", "background": "#f8f8f8", "borderRadius": "10px"}),
    dcc.Graph(id='cluster-radar'),
    html.Div(id='cluster-recommend'),
    html.Hr(),
    html.H4("Predicción de cluster para un nuevo cliente"),
    html.P("Selecciona los rangos que mejor describen al nuevo cliente. El sistema predice a qué cluster pertenece y lo explica."),
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='input-recency_days',
                options=[
                    {'label': '0-7 días', 'value': 4},
                    {'label': '8-30 días', 'value': 19},
                    {'label': '31-90 días', 'value': 60},
                    {'label': '91-180 días', 'value': 135},
                    {'label': '181-365+ días', 'value': 273}
                ],
                placeholder='Días desde última compra'
            )
        ], width=2),
        dbc.Col([
            dcc.Dropdown(
                id='input-frequency',
                options=[
                    {'label': '1 compra', 'value': 1},
                    {'label': '2-3 compras', 'value': 2.5},
                    {'label': '4-6 compras', 'value': 5},
                    {'label': '7-15 compras', 'value': 11},
                    {'label': '16+ compras', 'value': 20}
                ],
                placeholder='Cantidad de compras'
            )
        ], width=2),
        dbc.Col([
            dcc.Dropdown(
                id='input-monetary',
                options=[
                    {'label': '<$100', 'value': 50},
                    {'label': '$100-300', 'value': 200},
                    {'label': '$301-700', 'value': 500},
                    {'label': '$701-1500', 'value': 1100},
                    {'label': '>$1500', 'value': 2000}
                ],
                placeholder='Gasto total ($)'
            )
        ], width=2),
        dbc.Col([
            dcc.Dropdown(
                id='input-n_metodos_pago',
                options=[
                    {'label': '1', 'value': 1},
                    {'label': '2', 'value': 2},
                    {'label': '3 o más', 'value': 3}
                ],
                placeholder='# métodos de pago usados'
            )
        ], width=2),
    ], className="g-2"),
    html.Br(),
    html.Button('Predecir cluster', id='predict-btn', n_clicks=0, className="btn btn-primary"),
    html.Div(id='prediction-output', style={'fontWeight': 'bold', 'paddingTop': '1em'}),
    html.Hr(),
    html.H4("Comparativa de KPIs entre clusters"),
    dcc.Graph(figure=fig_kpi),
    html.H4("Evolución temporal del tamaño de los clusters"),
    dcc.Graph(figure=fig_evol),
    html.Hr(),
    html.H4("Detalle filtrable de clientes por cluster"),
    html.P("Tabla interactiva: puedes filtrar por cluster y ver los valores principales de cada cliente. También puedes exportar la tabla a CSV."),
    dcc.Dropdown(
        id='dropdown-cluster',
        options=[{"label": cluster_names[c], "value": str(c)} for c in sorted(full_df['cluster'].unique())],
        multi=True, value=[], clearable=True, placeholder='Filtrar por cluster...'
    ),
    html.Button("Exportar a CSV", id="export-btn", n_clicks=0, style={"marginLeft":10}),
    dcc.Download(id="download-clients"),
    dash_table.DataTable(
        id='clientes-table',
        columns=[{"name": i.replace('_z',' (Z)').replace('_',' ').title(), "id": i} for i in ["recency_z","frequency_z","monetary_z","payment_entropy"]+list(customer_topics.columns)+["cluster"]],
        data=full_df.reset_index().round(2).to_dict('records'),
        page_size=12, filter_action="native", sort_action="native", style_table={'overflowX': 'auto'}, style_cell={'textAlign': 'center'}
    ),
], fluid=True)

## 8. Callbacks Dash: lógica interactiva (explicación, radar, predicción y exportación)

In [9]:
@app.callback(
    [Output('cluster-desc-panel', 'children'),
     Output('cluster-radar', 'figure'),
     Output('cluster-recommend', 'children')],
    [Input('cluster-desc-dropdown', 'value')]
)
def update_desc_panel(cluster):
    return (
        cluster_summary(cluster, cluster_profile, top_words_topic, cluster_names, topic_explainer),
        radar_cluster(cluster, cluster_profile),
        cluster_recommendation(cluster)
    )
@app.callback(
    Output('prediction-output', 'children'),
    Input('predict-btn', 'n_clicks'),
    State('input-recency_days', 'value'),
    State('input-frequency', 'value'),
    State('input-monetary', 'value'),
    State('input-n_metodos_pago', 'value')
)
def predict_cluster_dropdowns(n_clicks, rec_days, freq, monet, n_methods):
    if n_clicks > 0 and all(v is not None for v in [rec_days, freq, monet, n_methods]):
        arr = np.zeros((1,3))
        arr[0,0] = rec_days
        arr[0,1] = freq
        arr[0,2] = monet
        recency_val, frequency_val, monetary_val = scaler_rfm.transform(arr)[0]
        probs = np.full(n_methods, 1/n_methods)
        payment_entropy_val = entropy(probs, base=2)
        topics_vals = [0]*K
        X_new = np.array([[recency_val, frequency_val, monetary_val, payment_entropy_val] + topics_vals])
        cluster_pred = kmeans.predict(X_new)[0]
        return cluster_summary(cluster_pred, cluster_profile, top_words_topic, cluster_names, topic_explainer) + "\n" + cluster_recommendation(cluster_pred).children[1].children
    return ""
@app.callback(
    [Output('clientes-table', 'data'),
     Output('download-clients', 'data')],
    [Input('dropdown-cluster', 'value'),
     Input('export-btn', 'n_clicks')],
    [State('clientes-table', 'data')]
)
def update_table_export(selected_clusters, n_clicks, table_data):
    df = full_df.reset_index().round(2)
    if selected_clusters:
        df = df[df['cluster'].astype(str).isin(selected_clusters)]
    data = df[["recency_z","frequency_z","monetary_z","payment_entropy"]+list(customer_topics.columns)+["cluster"]].to_dict('records')
    ctx = dash.callback_context
    if ctx.triggered and ctx.triggered[0]['prop_id'].startswith('export-btn') and n_clicks>0:
        return data, dcc.send_data_frame(df.to_csv, "clientes_segmentados.csv", index=False)
    return data, dash.no_update
# Para correr el dashboard en Jupyter Lab, usa un puerto libre:
# app.run(debug=True, port=8052)

In [10]:
app.run(debug=True, port=8052)